<a href="https://colab.research.google.com/github/xSupernovaa/NLP-Topic-Modelling/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Data loading
from google.colab import drive
import zipfile
import pandas as pd
# Preprocessing
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))
# Feature extraction
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaModel


# Util
from tqdm import tqdm
from pprint import pprint



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [43]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
DATA_PATH = "drive/MyDrive/articles1.csv.zip"

In [45]:
# Extract the CSV file from the zip archive
with zipfile.ZipFile(DATA_PATH, "r") as zip_ref:
    zip_ref.extractall()

# Load the CSV data into a Pandas DataFrame
df = pd.read_csv("articles1.csv")

In [46]:
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [47]:
# Extract title and content as its preprocessing will differ from other features
articles =  df['title'] + ' ' + df['content']

In [48]:
articles[0]

'House Republicans Fret About Winning Their Health Care Suit - The New York Times WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporari

In [49]:
def preprocess_articles(articles):

  lemmatizer = WordNetLemmatizer()

  # Remove punctuations and convert text to lowercase
  words = gensim.utils.simple_preprocess(articles, deacc=True)
    
  # Remove stop words & punctuation 
  words = [word for word in words if word not in stop_words and word not in string.punctuation]
  # Lemmatize
  words = [lemmatizer.lemmatize(word) for word in words]

  # remove all non-word and non-space
  # words = re.sub(r'[^\w\s]', '', words) 

  return words


In [50]:
preprocessed_articles = articles.map(preprocess_articles)

In [51]:
preprocessed_articles[0]

['house',
 'republican',
 'fret',
 'winning',
 'health',
 'care',
 'suit',
 'new',
 'york',
 'time',
 'washington',
 'congressional',
 'republican',
 'new',
 'fear',
 'come',
 'health',
 'care',
 'lawsuit',
 'obama',
 'administration',
 'might',
 'win',
 'incoming',
 'trump',
 'administration',
 'could',
 'choose',
 'longer',
 'defend',
 'executive',
 'branch',
 'suit',
 'challenge',
 'administration',
 'authority',
 'spend',
 'billion',
 'dollar',
 'health',
 'insurance',
 'subsidy',
 'american',
 'handing',
 'house',
 'republican',
 'big',
 'victory',
 'issue',
 'sudden',
 'loss',
 'disputed',
 'subsidy',
 'could',
 'conceivably',
 'cause',
 'health',
 'care',
 'program',
 'implode',
 'leaving',
 'million',
 'people',
 'without',
 'access',
 'health',
 'insurance',
 'republican',
 'prepared',
 'replacement',
 'could',
 'lead',
 'chaos',
 'insurance',
 'market',
 'spur',
 'political',
 'backlash',
 'republican',
 'gain',
 'full',
 'control',
 'government',
 'stave',
 'outcome',
 'repu

In [52]:
def extract_features(articles, method='tf-idf'):
    if method == 'tf-idf':
        vectorizer = TfidfVectorizer()
        feature_vectors = vectorizer.fit_transform(articles)
        feature_names = vectorizer.get_feature_names_out()
    elif method == 'countvectorizer':
        vectorizer = CountVectorizer()
        feature_vectors = vectorizer.fit_transform(articles)
        feature_names = vectorizer.get_feature_names_out()
    else:
        raise ValueError('Invalid method specified.')
    # TODO:
      # Add word2vec
    
    return feature_vectors, feature_names

In [53]:
# create Dictionary from preprocessed articles
dictionary = Dictionary(preprocessed_articles)

# bag of words corpus from dictionary
corpus_bow = [dictionary.doc2bow(article) for article in preprocessed_articles]

In [54]:
# create TF-IDF Model from corpus
tfidf = TfidfModel(corpus_bow)

# weighted tfidf corpus
corpus_tfidf = tfidf[corpus_bow]

In [55]:
# Apply LDA with 10 topics
lda_model = LdaModel(corpus=corpus_bow,
                     id2word=dictionary,
                     num_topics=10,
                     passes=10,
                     iterations=100)

In [56]:
topics = lda_model.show_topics(num_topics=10, num_words=10)
for topic in topics:
    print(topic)

(0, '0.017*"trump" + 0.016*"said" + 0.014*"president" + 0.009*"obama" + 0.008*"u" + 0.008*"house" + 0.007*"russia" + 0.007*"state" + 0.007*"would" + 0.007*"administration"')
(1, '0.010*"said" + 0.008*"one" + 0.007*"like" + 0.006*"time" + 0.006*"year" + 0.005*"first" + 0.004*"get" + 0.004*"new" + 0.004*"say" + 0.004*"day"')
(2, '0.022*"news" + 0.017*"medium" + 0.016*"twitter" + 0.013*"facebook" + 0.012*"breitbart" + 0.010*"show" + 0.008*"video" + 0.007*"story" + 0.006*"time" + 0.006*"post"')
(3, '0.021*"said" + 0.018*"police" + 0.011*"officer" + 0.010*"court" + 0.007*"law" + 0.007*"state" + 0.007*"case" + 0.006*"gun" + 0.005*"according" + 0.005*"told"')
(4, '0.046*"trump" + 0.017*"clinton" + 0.014*"said" + 0.013*"republican" + 0.009*"campaign" + 0.009*"donald" + 0.007*"president" + 0.007*"would" + 0.006*"hillary" + 0.006*"say"')
(5, '0.016*"country" + 0.010*"china" + 0.008*"government" + 0.008*"united" + 0.007*"said" + 0.007*"state" + 0.007*"party" + 0.006*"would" + 0.006*"world" + 0.00

In [58]:
# Apply LDA with TF-IDF and 15 topics
lda_model_tfidf = LdaModel(corpus_tfidf,
                           id2word=dictionary,
                           num_topics=15,
                           passes=10,
                           iterations=100)

In [59]:
# Print the top 10 topics
topics = lda_model_tfidf.show_topics(num_topics=15, num_words=20)
for topic in topics:
    print(topic)

(0, '0.005*"duterte" + 0.001*"rodrigo" + 0.001*"davao" + 0.000*"bruni" + 0.000*"vigilante" + 0.000*"matobato" + 0.000*"bunker" + 0.000*"andanar" + 0.000*"lima" + 0.000*"mindanao" + 0.000*"filipino" + 0.000*"marcos" + 0.000*"dd" + 0.000*"blurring" + 0.000*"dela" + 0.000*"leila" + 0.000*"poling" + 0.000*"karaoke" + 0.000*"narcostate" + 0.000*"narcostates"')
(1, '0.003*"fiorina" + 0.002*"pc" + 0.002*"vr" + 0.002*"griffin" + 0.001*"colbert" + 0.001*"carly" + 0.001*"mcmullin" + 0.001*"crutcher" + 0.001*"oculus" + 0.001*"letterman" + 0.001*"tattoo" + 0.001*"depp" + 0.001*"weaver" + 0.001*"headset" + 0.001*"dickerson" + 0.001*"schwarzenegger" + 0.001*"bernstein" + 0.000*"sayyaf" + 0.000*"pirate" + 0.000*"brat"')
(2, '0.002*"bolton" + 0.001*"marlow" + 0.001*"zimmerman" + 0.001*"cpac" + 0.001*"lauer" + 0.001*"gaffney" + 0.001*"ricketts" + 0.000*"doll" + 0.000*"geller" + 0.000*"prostitution" + 0.000*"machar" + 0.000*"kiir" + 0.000*"mcconaughey" + 0.000*"gonzaga" + 0.000*"dodger" + 0.000*"juba" +